# **Download Data**

In [1]:
import pandas as pd
import requests
import time
import os
from datetime import datetime

In [15]:
def get_crypto_prices(crypto_id, currency="usd", days=365):

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {"vs_currency": currency, "days": days, "interval": "daily"}
    response = requests.get(url, params=params)

    # Si la respuesta es exitosa, se procesan los datos
    if response.status_code == 200:

        # Se convierte la respuesta a un DataFrame
        # data = response.json()["prices"]
        # df = pd.DataFrame(data, columns=["timestamp", "adjusted_price"])
        # df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

        data = response.json()
        df = pd.DataFrame(data)
        
        # Se guarda el archivo en la carpeta "crypto_data_YYYY-MM-DD"
        today = datetime.now().strftime("%Y-%m-%d")
        folder = f"crypto-prices-{today}-{days}"
        
        if not os.path.exists(folder):
            os.makedirs(folder)

        filename = f"{folder}/{crypto_id}.csv"
        df.to_csv(filename, index=False)

        print(f"Precios de {crypto_id} guardados en {filename}")        
        return None # No hay errores
    
    
    # Si la respuesta no es exitosa, se imprime un mensaje de error
    else:
        print(f"Error al obtener datos de {crypto_id}: {response.status_code}")
        
        if response.status_code == 429:
            print(f"Demasiadas peticiones. Esperando {response.headers['Retry-After']} segundos.")
            time.sleep(int(response.headers["Retry-After"]))
            get_crypto_prices(crypto_id, currency, days)
            
        return response.status_code

In [16]:
# Lista de 50 criptomonedas
cryptos = [
    "bitcoin", "ethereum", "tether", "binancecoin", "usd-coin",
    "ripple", "cardano", "solana", "dogecoin", "polkadot",
    "shiba-inu", "dai", "tron", "avalanche-2", "wrapped-bitcoin",
    "uniswap", "litecoin", "chainlink", "cosmos", "stellar",
    "monero", "algorand", "bitcoin-cash", "vechain", "filecoin",
    "internet-computer", "hedera", "decentraland", "the-sandbox", "tezos",
    "elrond-erd-2", "theta-token", "aave", "eos", "pancakeswap-token",
    "flow", "zcash", "maker", "klay-token", "neo",
    "iota", "fantom", "kusama", "waves", "dash",
    "synthetix-network-token", "compound", "chiliz", "enjincoin"
]

In [17]:
for crypto in cryptos:
    get_crypto_prices(crypto, days=365)

Precios de bitcoin guardados en crypto-prices-2024-12-26-365/bitcoin.csv
Precios de ethereum guardados en crypto-prices-2024-12-26-365/ethereum.csv
Precios de tether guardados en crypto-prices-2024-12-26-365/tether.csv
Precios de binancecoin guardados en crypto-prices-2024-12-26-365/binancecoin.csv
Precios de usd-coin guardados en crypto-prices-2024-12-26-365/usd-coin.csv
Error al obtener datos de ripple: 429
Demasiadas peticiones. Esperando 60 segundos.


KeyboardInterrupt: 

# **All in One**

In [10]:
csv_folder = "C:\\Heri\\GitHub\\Crypto\\02-Coingecko\\01-Prices\\crypto-prices-2024-12-26-365" # Carpeta donde están los CSV

In [11]:
all_data = {}  # Diccionario para almacenar datos de cada criptomoneda

for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        crypto_name = file.replace(".csv", "")  # Obtener nombre de la cripto
        file_path = os.path.join(csv_folder, file)
        df = pd.read_csv(file_path)  # Leer el CSV
        df["timestamp"] = pd.to_datetime(df["timestamp"])  # Asegurar formato datetime
        df = df.set_index("timestamp")  # Usar fecha como índice
        all_data[crypto_name] = df["adjusted_price"]  # Guardar precios ajustados

# Crear DataFrame con todos los datos
data_crypto = pd.DataFrame(all_data)

In [12]:
data_crypto.dropna(inplace=True)  # Eliminar filas con valores nulos

# **Create the average**

In [13]:
# Calculate the average of all columns for each row
data_crypto['average'] = data_crypto.mean(axis=1)

In [14]:
data_crypto.to_csv("data-crypto.csv")